In [1]:
import numpy as np
import pandas as pd

In [2]:
#remove duplicate rows
def remove_duplicate(np_f,np_g):
    unique_f = np.unique(np_f,axis=0)
    unique_g = np.unique(np_g,axis=0)
    return unique_f, unique_g

In [3]:
#remove super set
def remove_superset(np_arr):
    #sort the array by row sum(max to min)
    np_arr = sorted(np_arr, key=lambda x: sum(x),reverse=True)
    np_arr = np.asarray(np_arr)
    # store the row indexes to be deleted
    delete_rows = []
    for i in range(len(np_arr)):
        # find the column where the value is 0
        zero_index = np.where(np_arr[i] == 0)[0]
        if(len(zero_index) != 0):
            zero_cols = np_arr[:, np.array(zero_index)]
            rows_sum = np.sum(zero_cols, axis=1)
            # check if there exists another row where the value are all-zeros at these columns
            if np.count_nonzero(rows_sum == 0) > 1:
                delete_rows.append(i)
    np_arr = np.delete(np_arr,delete_rows,0) 
    return np_arr      

In [4]:
# remove_superset test sample:
np_arr = np.array([[0,0,0,0,0,0,0,0],
               [0,0,0,0,0,0,1,1]])
print(remove_superset(np_arr))

[[0 0 0 0 0 0 0 0]]


In [5]:
# display the data frame with column total and row total attributes
def display(f,g):
    f=pd.DataFrame(unique_f)
    f.loc['Column_Total']= f.sum(numeric_only=True, axis=0)
    fmax_col_index = np.argmax(np.array(f.loc['Column_Total']))
    f.loc[:,'Row_Total'] = f.sum(numeric_only=True,axis=1)
    fmax_row_value=max(f['Row_Total'][0:-1])
    df_f=f
    print(df_f)
    print("the variable that appears max in f is: X",fmax_col_index)

    g=pd.DataFrame(unique_g)
    g.loc['Column_Total']= g.sum(numeric_only=True, axis=0)
    gmax_col_index = np.argmax(np.array(g.loc['Column_Total']))
    g.loc[:,'Row_Total'] = g.sum(numeric_only=True,axis=1)
    gmax_row_value=max(g['Row_Total'][0:-1])
    df_g=g
    print(df_g)
    print("the variable that appears max in g is：X",gmax_col_index)
    return df_f,df_g

In [6]:
# pre-req
#np_f and np_g are numpy arrays
def check_pre(f,g):
# input: f, g -- numpy array
# 1. assume var in f and g are the same, and we check if # of columns in f == # of columns in g 
    if np.shape(f)[1] != np.shape(g)[1]: 
        print("not dual, # of columns in f != # of columns in g")
        return False
# 2.check if largest of sum_row in f <= # of rows in g, largest value of sum_row in g <= # of rows in f 
    fmax_sum_row = max(f.sum(axis=1))
    gmax_sum_row = max(g.sum(axis=1))
    if fmax_sum_row > np.shape(g[0]): 
        print("not dual, fmax_row_value > len(g)")
        return False
    
    if gmax_sum_row > np.shape(f[0]):
        print("not dual, gmax_row_value > len(f)")
        return False
# 3. sum 2^(-|c in f|)+sum 2^(-|c in g|) >=1, where c is each sum_row value(for loop)>=1
    sum_fg = 0
    f_row_sum_array = f.sum(axis=1)
    g_row_sum_array = g.sum(axis=1)
    
    for i in range(np.shape(f)[0]):
        sum_fg += 1/(2**(f_row_sum_array[i]))
    for i in range(np.shape(g)[0]):
        sum_fg += 1/(2**(g_row_sum_array[i])) 
    if sum_fg < 1:
        print("not dual, sum_fg < 1")
        return False
        
# 4. C ^ C' != empty
    # need to check if the one is in the same position
    #         import pdb;  pdb.set_trace()  
    for i in range(len(f)):
        for j in range(np.shape(f)[1]):
            if f[i][j] ^ g[i][j] == True:
                pass
            else:
                return True
        return False


In [7]:
#spiltting, 
#F0=rows that contains x, change the position of x to 0
#F1=rest of the rows
def split(f,g):
    # split function return F0, F1 and g0, g1
    f_sumcol=f.sum(axis=0)
    g_sumcol =g.sum(axis=0)
    index=np.argmax(f_sumcol+g_sumcol) #find the most freq var(which has the max total column sum in f and g )
    print(f'splitting variable is x{index+1}')
    #In case of multiple occurrences of the maximum values, argmax() makes sure the indices corresponding to the first occurrence are returned.
    # it is proved that the most frequent var has the frequency >=1/log2(|f|+|g|) #Thomas paper theorem 1.3.3
    f0=f[f[:,index]==1]  ## select rows where spliting var column is ==1
    f0[:,index]=0 ##change the  position of splitting var to 0
    f1=f[f[:,index]==0]
    #print(f0,f1)
    g0=g[g[:,index]==1]  ## select rows where spliting var column is ==1
    g0[:,index]=0 ##change the  position of splitting var to 0
    g1=g[g[:,index]==0]  
    return f0,f1,g0,g1

def check_base(f,g):
    #|F|=1, |G|=0 or |F|=0, |G|=1
    if np.shape(f)[0]==1 and np.shape(g)[0]==0:
        if f.sum(axis=1)==0:
            #print("passed 1/0")
            return True
        else:
            print("check_base0/1")
            return False
    elif np.shape(g)[0]==1 and np.shape(f)[0]==0:  
        if g.sum(axis=1)==0:
            #print("passed 0/1")
            return True
        else:
            print("check_base1/0")
            return False
    if (np.shape(f)[0]*np.shape(g)[0])==0:
        return True
    # |f|=1 and |g|=1: they have and only have 1 same var in the same position, then they are dual to each other 
    elif np.shape(f)[0]==1 and np.shape(g)[0] == 1:
            if f.sum(axis=1)==1 and g.sum(axis=1)==1:
                if np.array_equal(f, g):
                    #print("passed 1/1")
                    return True
                else:
                    print("check_base1/1 sum=1, but not equal")
                    return False
            else:
                print("check_base1/1")
                return False
            
    elif np.shape(f)[0]==1 and np.shape(g)[0]>0:
        # if |f|=1 and |g|=k, easy way to check_dual: sum all columns in g and see if == f
        if np.array_equal(f.sum(axis=0), g.sum(axis=0)):
            for i in range(len(f)):
                zero_index = np.where(f[i] == 1)[0]
            if (np.shape(f)[1]!=np.shape(g)[0]) or (np.shape(f)[1]!=np.shape(g)[1]):
                return False
            for i in zero_index:
                if g[i][i]!=1:
                    print("check_base k/1")
                    return False
            return True
        else:
            print("check_base 1/k")
            return False
    
    elif np.shape(g)[0]==1 and np.shape(f)[0]>0:
        # if |g|=1 and |f|=k, easy way to check_dual: sum all columns in f and see if == g
        if np.array_equal(g.sum(axis=0), f.sum(axis=0)):
            #record the 1 column index in f
            for i in range(len(g)):
                zero_index = np.where(g[i] == 1)[0]
            if (np.shape(g)[1]!=np.shape(f)[0]) or (np.shape(g)[1]!=np.shape(f)[1]):
                return False
            for i in zero_index:
                if f[i][i]!=1:
                    print("check_base k/1")
                    return False
            return True
        else:
            print("check_base k/1")
            return False

          
        
# e.g.: error_path = [path_0, path_1] = [['L', 'R'], ['L','L','L']]

def fk(f, g, path):
    f,g=remove_duplicate(f,g)
    f=remove_superset(f)
    g=remove_superset(g)
    if check_base(f, g)==True: 
        return True
    if check_base(f, g)==False:
        return False
    elif check_pre(f, g)==False:
        return False
    else:
        print(f"path: {path}")
        f0,f1,g0,g1 = split(f, g)
        import pdb; pdb.set_trace() # function to check each step
        a0=f1
        a1=np.concatenate((g0, g1),axis=0)
        b0=g1
        b1=np.concatenate((f0,f1), axis=0)
        path.append("L")
        fk(a0, a1, path)
        del path[len(path) - 1:]
        #path = path[0:-1]        
        path.append("R")
        fk(b0, b1, path)
        del path[len(path) - 1:]
        return True
  


In [8]:
# two dual matrices 
# np_f= np.array([[1,1,0,1,0,0,0],
#                    [0,1,1,0,1,0,0],
#                    [0,0,1,1,0,1,0],
#                    [0,0,0,1,1,0,1],
#                    [1,0,0,0,1,1,0],
#                    [0,1,0,0,0,1,1],
#                    [1,0,1,0,0,0,1]])

# np_g= np.array([[1,1,0,1,0,0,0],
#                    [0,0,1,0,1,0,0],
#                    [0,0,1,1,0,1,0],
#                    [0,0,0,1,1,0,1],
#                    [1,0,0,0,1,1,0],
#                    [0,1,0,0,0,1,1],
#                    [1,0,1,0,0,0,1]])

#f2, g2
np_f= np.array([[1,0,1,0,0,0,0,0],
                [1,0,0,1,0,0,0,0],
                [0,1,1,0,0,0,0,0],
                [0,1,0,1,0,0,0,0],
                [0,0,0,0,1,0,1,0],
                [0,0,0,0,1,0,0,1],
                [0,0,0,0,0,1,1,0],
                [0,0,0,0,0,1,0,1]])

np_g= np.array([[1,1,0,0,0,0,1,1],
                [0,0,1,1,1,1,0,0],
                [1,1,0,0,1,1,0,0],
                [0,0,1,1,0,0,1,1]])

# np_f= np.array([[1,0,0,0,0,0,0],
#                 [0,1,0,0,0,0,0],
#                 [0,0,1,0,0,1,0],
#                 [0,0,0,1,0,0,0],
#                 [1,0,0,0,1,0,0],
#                 [0,1,0,0,0,1,0],
#                 [1,0,0,0,0,0,1]])
# np_g= np.array([[1,1,1,1,1,0,0]]): 

In [9]:
# test fk
path=[]
res=fk(np_f, np_g, path)
if res==True:
    print("they are dual to each other")
else:
    print("they are not dual")

path: []
splitting variable is x1
> /var/folders/1l/rydn75952kx389k4dthmtmqh0000gn/T/ipykernel_26068/349143807.py(103)fk()
    101         f0,f1,g0,g1 = split(f, g)
    102         import pdb; pdb.set_trace() # function to check each step
--> 103         a0=f1
    104         a1=np.concatenate((g0, g1),axis=0)
    105         b0=g1

ipdb> c
path: ['L']
splitting variable is x2
> /var/folders/1l/rydn75952kx389k4dthmtmqh0000gn/T/ipykernel_26068/349143807.py(103)fk()
    101         f0,f1,g0,g1 = split(f, g)
    102         import pdb; pdb.set_trace() # function to check each step
--> 103         a0=f1
    104         a1=np.concatenate((g0, g1),axis=0)
    105         b0=g1

ipdb> c
path: ['L', 'L']
splitting variable is x5
> /var/folders/1l/rydn75952kx389k4dthmtmqh0000gn/T/ipykernel_26068/349143807.py(103)fk()
    101         f0,f1,g0,g1 = split(f, g)
    102         import pdb; pdb.set_trace() # function to check each step
--> 103         a0=f1
    104         a1=np.concatenate((g0, g1)

ipdb> c
path: ['R']
splitting variable is x3
> /var/folders/1l/rydn75952kx389k4dthmtmqh0000gn/T/ipykernel_26068/349143807.py(103)fk()
    101         f0,f1,g0,g1 = split(f, g)
    102         import pdb; pdb.set_trace() # function to check each step
--> 103         a0=f1
    104         a1=np.concatenate((g0, g1),axis=0)
    105         b0=g1

ipdb> c
path: ['R', 'R']
splitting variable is x4
> /var/folders/1l/rydn75952kx389k4dthmtmqh0000gn/T/ipykernel_26068/349143807.py(103)fk()
    101         f0,f1,g0,g1 = split(f, g)
    102         import pdb; pdb.set_trace() # function to check each step
--> 103         a0=f1
    104         a1=np.concatenate((g0, g1),axis=0)
    105         b0=g1

ipdb> c
path: ['R', 'R', 'L']
splitting variable is x5
> /var/folders/1l/rydn75952kx389k4dthmtmqh0000gn/T/ipykernel_26068/349143807.py(103)fk()
    101         f0,f1,g0,g1 = split(f, g)
    102         import pdb; pdb.set_trace() # function to check each step
--> 103         a0=f1
    104         a1=np